# BTS MBDS DDB A3
## Part B: Feature Engineering

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Jedi not working
%config Completer.use_jedi = False

In [ ]:
df_prev = pd.read_csv('data/previous_application.csv')

In [ ]:
df_prev.head()

In [ ]:
df_prev = df_prev[['SK_ID_CURR',
          'AMT_ANNUITY',
        'AMT_APPLICATION' ,
           'AMT_CREDIT',
        'AMT_DOWN_PAYMENT',
         'AMT_GOODS_PRICE',
          'DAYS_DECISION',
       'HOUR_APPR_PROCESS_START',
      'RATE_DOWN_PAYMENT',
        'SK_ID_PREV']]

In [ ]:
df_prev["NEW_APP_CREDIT_RATE"] = df_prev['AMT_APPLICATION'] / df_prev['AMT_CREDIT']
df_prev["NEW_AMP_PAYMENT_RATE"] = df_prev['AMT_DOWN_PAYMENT'] / df_prev['AMT_CREDIT']
df_prev["NEW_APP_GOODS_RATE"] = df_prev['AMT_APPLICATION'] / df_prev['AMT_DOWN_PAYMENT']
df_prev["NEW_CREDIT_GOODS_RATE"] = df_prev['AMT_APPLICATION'] / df_prev['NEW_APP_CREDIT_RATE']
df_prev["NEW_RETURN_DAY"] = df_prev['AMT_ANNUITY'] / df_prev['AMT_CREDIT']
df_prev["NEW_DAYS_TERMINATION_OFF"] = df_prev['AMT_ANNUITY'] / df_prev['DAYS_DECISION']
df_prev["NEW_END_DIFF"] = df_prev['DAYS_DECISION'] / df_prev['AMT_ANNUITY']
df_prev["NEW_APP_CREDIT_RATE_RATIO"] = df_prev['AMT_CREDIT'] / df_prev['RATE_DOWN_PAYMENT']
df_prev["NEW_DAYS_DUE_DIFF"] = df_prev['DAYS_DECISION'] / df_prev['AMT_ANNUITY']
num_agg = {
        'SK_ID_PREV' : 'count',
        'AMT_ANNUITY': ['min', 'max', 'median', 'mean'], 
        'AMT_APPLICATION': ['min', 'max', 'mean', 'median'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'median'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'median'],
        'DAYS_DECISION': ['min', 'max', 'mean','median'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean','median'],     
        "NEW_APP_CREDIT_RATE": ['min', 'max', 'mean','median'],
        "NEW_AMP_PAYMENT_RATE": ['min', 'max', 'mean','median'],
        "NEW_APP_GOODS_RATE": ['min', 'max', 'mean','median'],
        "NEW_CREDIT_GOODS_RATE": ['min', 'max', 'mean','median'],
        "NEW_RETURN_DAY": ['min', 'max', 'mean','median'],
        "NEW_DAYS_TERMINATION_OFF": ['min', 'max', 'mean','median'],
        "NEW_END_DIFF": ['min', 'max', 'mean','median'],
        "NEW_APP_CREDIT_RATE_RATIO": ['min', 'max', 'mean','median'],
        "NEW_DAYS_DUE_DIFF": ['min', 'max', 'mean','median'],
    }

In [ ]:
df_prev_agg = df_prev.groupby(by='SK_ID_CURR').agg({**num_agg})

In [ ]:
df_prev_agg.head().T

## Part C: Logistic Regression

In [1]:
# imports
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix  , classification_report ,accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%config Completer.use_jedi = False
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('data/application_train.csv')
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, df['TARGET'], test_size=0.2)
print(X_train.shape, X_test.shape)

(246008, 122) (61503, 122)


In [4]:
X_train = X_train.select_dtypes(exclude=['float64' , 'int64'])
X_test = X_test.select_dtypes(exclude=['float64' , 'int64'])
print(X_train.shape, X_test.shape)

(246008, 16) (61503, 16)


In [5]:
X_train.isna().sum().sort_values(ascending=False)

FONDKAPREMONT_MODE            168337
WALLSMATERIAL_MODE            125213
HOUSETYPE_MODE                123620
EMERGENCYSTATE_MODE           116761
OCCUPATION_TYPE                77149
NAME_TYPE_SUITE                 1042
NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                       0
FLAG_OWN_REALTY                    0
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
WEEKDAY_APPR_PROCESS_START         0
ORGANIZATION_TYPE                  0
dtype: int64

In [6]:
X_train = X_train.drop(['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE','HOUSETYPE_MODE','EMERGENCYSTATE_MODE','OCCUPATION_TYPE','NAME_TYPE_SUITE'], axis=1)
X_test = X_test.drop(['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE','HOUSETYPE_MODE','EMERGENCYSTATE_MODE','OCCUPATION_TYPE','NAME_TYPE_SUITE'], axis=1)
print(X_train.shape, X_test.shape)

(246008, 10) (61503, 10)


In [7]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
print(X_train.shape, X_test.shape)

(246008, 99) (61503, 99)


In [8]:
log_clf = LogisticRegression(penalty='l2',solver='lbfgs',max_iter=500)

In [9]:
log_clf.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [10]:
log_clf.score(X_test, y_test)

0.9184104840414289